In [57]:
from CharRNN import CharRNN
import pandas as pd
import tensorflow as tf
from utils import *
from tqdm import trange
from rdkit import Chem
from rdkit.Chem import Draw
from chembl_webresource_client.new_client import new_client

In [2]:
rnn_cell_size = 64
num_layers = 1
embedding_size = 32

split_size = 30
batchsize = 100
epochs = 20
stride=1

num_features = 3

In [3]:
sess = tf.Session()

In [4]:
model = CharRNN(sess, len(elems)+num_features, len(elems), embedding_size, 
                rnn_cell_size, num_layers, len(elems)-1, sess_chkpt='tmp/0_70.chkpt')

INFO:tensorflow:Restoring parameters from tmp/0_70.chkpt


In [51]:
chems = []
for i in trange(100):
    chem = Chem.MolFromSmiles(generate_new(model, [0.1, 0.3, 0]))
    if chem is not None:
        chems.append(chem)


100%|██████████| 100/100 [00:03<00:00, 29.61it/s]


In [69]:
def get_bondings(target_chembl):

    target = new_client.target
    activity = new_client.activity
    molecule = new_client.molecule

    activity = activity.filter(target_chembl_id=target_chembl).filter(standard_type="IC50")

    molecules = []

    for act in tqdm_notebook(activity):
        id = act['molecule_chembl_id']
        molec = molecule.get(id)
        
        if (act['standard_value'] is not None and 
            molec['molecule_properties']['acd_logd'] is not None and
            molec['molecule_properties']['acd_logp'] is not None):
            
            molecules.append({
                'canonical_smiles': molec['molecule_structures']['canonical_smiles'],
                'acd_logd': float(molec['molecule_properties']['acd_logd']),
                'acd_logp': float(molec['molecule_properties']['acd_logp']),
                'affinity': float(act['standard_value'])

            })
    return molecules

In [70]:
target_id = 'CHEMBL2074'

In [71]:
bondings = get_bondings(target_id)

In [87]:
bondings[110]

{'acd_logd': 2.95,
 'acd_logp': 3.08,
 'affinity': 100000.0,
 'canonical_smiles': 'OC[C@H]1[C@@H](O)[C@H](O)[C@@H](O)CN1CCCCCOCc2ccccc2c3ccccc3'}